In [1]:
import numpy as np
from itertools import chain, combinations
import sys
import networkx as nx

np.set_printoptions(threshold=sys.maxsize)

In [2]:
def powerset(iterable):
    "powerset([1,2,3]) → () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [3]:
def splitStructure(structure):
    twos = []
    threes = []
    for auth in structure:
        if len(auth) == 2:
            twos.append(auth)
        else:
            threes.append(auth)

    return twos,threes

In [4]:
def flatten(lst):
    l = []
    for elt in lst:
        t = type(elt)
        if t is tuple or t is list:
            for elt2 in flatten(elt):
                l.append(elt2)
        else:
            l.append(elt)
    return sorted(set(l))

In [5]:
def isAuth(P,structure):
    for minAuth in structure:
        if set(minAuth).issubset(set(P)):
            return True
    return False

In [6]:
def buildD1(structure, p):
    P = list(range(1,p+1))
    D1 = list(powerset(P))

    for subset in D1[:]:
        if isAuth(subset,structure) or len(subset) == 0:
            D1.remove(subset)

    for unauth in D1[:]:
        for set2 in D1:
            if set2 != unauth and set(unauth).issubset(set(set2)):
                D1.remove(unauth)
                break
        else:
            continue
    return D1

In [7]:
def NEW_buildD1(structure, p):
    twos, threes = splitStructure(structure)

    P = flatten(structure)

    G = nx.Graph()
    G = nx.Graph()
    G.add_nodes_from(P)
    G.add_edges_from(twos)
    G_complement = nx.complement(G)

    D1 = list(nx.find_cliques(G_complement))

    for auth in threes:
        for IS in D1[:]:
            if set(auth).issubset(set(IS)):
                D1.remove(IS)

    return D1



In [8]:
def checkD1(D1):
    for [P1,P2] in combinations(D1,2):
        if len(set(P1) & set(P2)) > 1:
            return False
    return True

In [9]:
def listIntersection(lst1,lst2):
    return set([tuple(sorted(ele)) for ele in lst1]) & set([tuple(sorted(ele)) for ele in lst2])

In [10]:
def findFailingD1(D1):
    fail = set()
    for [P1,P2] in combinations(D1,2):
        if len(set(P1) & set(P2)) > 1:
            fail.add(tuple(set(P1) & set(P2)))
    return sorted(list(fail))

In [11]:
def buildD2(structure):
    D2 = []

    for P in structure:
        if len(P) == 2:
            D2.append(P)

    P = flatten(D2)

    G = nx.Graph()
    G.add_nodes_from(P)
    G.add_edges_from(D2)

    return list(nx.find_cliques(G))

In [12]:
def checkD2(D2):
    for [P1,P2] in combinations(D2,2):
        if len(set(P1) & set(P2)) > 0:
            return False
    return True

In [13]:
def checkCondtions(structure, p):
    D1 = buildD1(structure,p)
    D2 = buildD2(structure)

    if checkD1(D1) and checkD2(D2):
        return True
    
    return False

In [14]:
def buildT2rec(sp,p,l):
    T2 = []

    if p - sp < 1 or l < 2 or sp + l > p + 1:
        return T2
    
    structure = list(combinations(list(range(sp,sp+l)),2))
    start = 2
    for nl in range(start,l+1):
        recur = buildT2rec(sp+l,p,nl)
        if len(recur) > 0:
            for smaller in recur:
                temp = structure[:]
                for P in smaller:
                    temp.append(P)
                T2.append(temp)    
    T2.append(structure)

    return T2

In [15]:
def buildT2(p):
    T2 = []
    for l in range(2,p+1):
        T2a = buildT2rec(1,p,l)
        for elt in T2a:
            if elt not in T2:
                T2.append(elt)
    T2.append([])
    return T2

In [16]:
def noSubset(P,pairs):
    for pair in pairs:
        if set(pair).issubset(P):
            return False
    return True

In [17]:
def fixD1(structure,p):
    removedSets= []
    if len(structure) == 0 and p > 2:
        structure.append((1,2,3))

    D1 = buildD1(structure,p)

    while not checkD1(D1):
        removedSet = False
        failing = findFailingD1(D1)
        for P in D1:
            for fail in failing:
                if set(fail).issubset(set(P)) and noSubset(P,removedSets):
                    if len(P) > 3:
                        structure.append((P[0],P[1],P[2]))
                    else:
                        structure.append(P)
                    removedSets = removedSets + list(combinations(P,2))
                    removedSet = True
                    break
            else:
                continue
        if removedSet:
            D1 = buildD1(structure,p)
        else:
            removedSets = []

    return structure

In [18]:
AccessStr = [(1,2),(3,4),(5,6),(1,3,5),(1,4,6),(2,3,6),(2,4,5)]
# AccessStr = [(1,2),(3,4),(5,6)]
# AccessStr = [(1,2),(3,4)] # Works with q=4?
# AccessStr = [(1,2),(2,3),(1,3),(3,4,5),(2,4,5)]
# AccessStr = [(1, 2), (3, 4), (1, 3, 5), (2, 4, 5)] # q = 5?
# AccessStr = [(1, 2, 3)]
# AccessStr = [(1, 2, 3), (4, 5)]
# AccessStr = [(1,2),(3,4),(1,2,3),(1,2,4),(1,3,4),(2,3,4)]
# AccessStr = [(1,2,3),(1,5,7),(1,6,8),(2,4,7),(2,6,9),(3,4,8),(3,5,9),(4,5,6)]
# AccessStr = [(1,2),(2,3),(3,4)]

p = 6

In [19]:
list(NEW_buildD1(AccessStr,p))

[[1, 3, 6], [1, 4, 5], [2, 3, 5], [2, 4, 6]]

In [20]:
fixD1(AccessStr,p)

[(1, 2), (3, 4), (5, 6), (1, 3, 5), (1, 4, 6), (2, 3, 6), (2, 4, 5)]

In [21]:
def findRealizable(p):
    real = []
 
    for structure in buildT2(p):
        D1 = NEW_buildD1(structure,p)
        real.append(fixD1(structure,p))
    
    return real


In [22]:
buildD1(AccessStr,6)

[(1, 3, 6), (1, 4, 5), (2, 3, 5), (2, 4, 6)]

In [23]:
findRealizable(4)

[[(1, 2), (3, 4)],
 [(1, 2), (1, 3, 4)],
 [(1, 2), (1, 3), (2, 3)],
 [(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)],
 [(1, 2, 3), (1, 2, 4), (1, 3, 4)]]

In [24]:
findRealizable(5)

[[(1, 2), (3, 4), (1, 3, 5), (2, 4, 5)],
 [(1, 2), (1, 3, 4), (1, 3, 5), (1, 4, 5)],
 [(1, 2), (1, 3), (2, 3), (4, 5)],
 [(1, 2), (1, 3), (2, 3), (1, 4, 5), (2, 4, 5)],
 [(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)],
 [(1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (2, 3),
  (2, 4),
  (2, 5),
  (3, 4),
  (3, 5),
  (4, 5)],
 [(1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 3, 4), (1, 3, 5), (1, 4, 5)]]

In [ ]:
findRealizable(6)

[[(1, 2), (3, 4), (5, 6), (1, 3, 5), (1, 4, 6), (2, 3, 6), (2, 4, 5)],
 [(1, 2),
  (3, 4),
  (1, 3, 5),
  (1, 3, 6),
  (2, 4, 5),
  (2, 4, 6),
  (1, 4, 5),
  (1, 4, 6),
  (2, 3, 5),
  (1, 5, 6),
  (2, 3, 6),
  (2, 5, 6),
  (3, 5, 6)],
 [(1, 2), (1, 3, 4), (1, 3, 5), (1, 3, 6), (1, 4, 5), (1, 4, 6), (1, 5, 6)],
 [(1, 2),
  (1, 3),
  (2, 3),
  (4, 5),
  (1, 4, 6),
  (2, 5, 6),
  (1, 5, 6),
  (2, 4, 6),
  (3, 4, 6)],
 [(1, 2), (1, 3), (2, 3), (4, 5), (4, 6), (5, 6)],
 [(1, 2),
  (1, 3),
  (2, 3),
  (1, 4, 5),
  (1, 4, 6),
  (1, 5, 6),
  (2, 4, 5),
  (2, 4, 6),
  (2, 5, 6)],
 [(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4), (5, 6)],
 [(1, 2),
  (1, 3),
  (1, 4),
  (2, 3),
  (2, 4),
  (3, 4),
  (1, 5, 6),
  (2, 5, 6),
  (3, 5, 6)],
 [(1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (2, 3),
  (2, 4),
  (2, 5),
  (3, 4),
  (3, 5),
  (4, 5)],
 [(1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (1, 6),
  (2, 3),
  (2, 4),
  (2, 5),
  (2, 6),
  (3, 4),
  (3, 5),
  (3, 6),
  (4, 5),
  (4, 6),
  (5, 6)],
 [(1, 2, 3)

In [26]:
AccessStr = [(1,2,3),(1, 2, 4), (1, 2, 5), (1, 3, 4), (1, 3, 5),(2,3,5),(2,4,5),(3,4,5)]

In [27]:
# got this from stack exchange
def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = np.array2string(a, max_line_width=np.infty).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r' \\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

In [28]:
def toString(minQ):
    if len(minQ) == 0:
        return ' '
    counter = 0
    sets = '$'
    for qual in minQ:
        sets += r'\{'
        for elt in qual:
            sets += 'p_' + str(elt) + ','
        sets = sets[:-1] + r'\},'
        counter += 1

    return sets[:-1]+'$'

In [29]:
def makeTable(meetConditions):
    table = []
    for structure in meetConditions:
        size2 = []
        size3 = []
        for minQ in structure:
            if len(minQ) == 2:
                size2.append(minQ)
            if len(minQ) == 3:
                size3.append(minQ)
        table += [' ' + ', '.join(toString(size2).split()) + ' & ' + ', '.join(toString(size3).split()) + r' &  \\ \hline'] 

    return '\n'.join(table)

In [30]:
def newFindD1(structure: list, D1: list, A: list):
    p = max(flatten(structure))

    if isAuth(A,structure):
        A.remove(max(A))

        for x in (x for x in range(1,p+1) if x not in A):
            if not isAuth(A+[x],structure):
                return D1
            
        if A not in D1:
            return D1+[A]
        else:
            return D1
        
    if p in A:
        for x in (x for x in range(1,p+1) if x not in A):
            if not isAuth(A+[x],structure):
                return D1
            
        if A not in D1:
            return D1+[A]
        else:
            return D1

    if len(A) == 0:
        for x in range(1,p+1):
            D1 = newFindD1(structure,D1,A+[x])
    else:
        for x in (x for x in range(1,p+1) if x > max(A)):
            D1 = newFindD1(structure,D1,A+[x])

    return D1

In [31]:
def newFindD1(structure: list, A: list, D1: list):
    flag = True
    n = max(flatten(structure))
    if len(A) == 0:
        max_p = 0
    else:
        max_p = max(A)

    for x in range(max_p+1,n+1):
        if not isAuth(A + [x], structure):
            D1 = newFindD1(structure, A + [x], D1)
            flag = False

    if flag:
        for x in [val for val in range(1,max_p+1) if val not in A]:
            if not isAuth(A + [x], structure):
                return D1
        
        D1 = D1 + [A]

    return D1

In [32]:
findRealizable(6)

[[(1, 2), (3, 4), (5, 6), (1, 3, 5), (1, 4, 6), (2, 3, 6), (2, 4, 5)],
 [(1, 2),
  (3, 4),
  (1, 3, 5),
  (1, 3, 6),
  (2, 4, 5),
  (2, 4, 6),
  (1, 4, 5),
  (1, 4, 6),
  (2, 3, 5),
  (1, 5, 6),
  (2, 3, 6),
  (2, 5, 6),
  (3, 5, 6)],
 [(1, 2), (1, 3, 4), (1, 3, 5), (1, 3, 6), (1, 4, 5), (1, 4, 6), (1, 5, 6)],
 [(1, 2),
  (1, 3),
  (2, 3),
  (4, 5),
  (1, 4, 6),
  (2, 5, 6),
  (1, 5, 6),
  (2, 4, 6),
  (3, 4, 6)],
 [(1, 2), (1, 3), (2, 3), (4, 5), (4, 6), (5, 6)],
 [(1, 2),
  (1, 3),
  (2, 3),
  (1, 4, 5),
  (1, 4, 6),
  (1, 5, 6),
  (2, 4, 5),
  (2, 4, 6),
  (2, 5, 6)],
 [(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4), (5, 6)],
 [(1, 2),
  (1, 3),
  (1, 4),
  (2, 3),
  (2, 4),
  (3, 4),
  (1, 5, 6),
  (2, 5, 6),
  (3, 5, 6)],
 [(1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (2, 3),
  (2, 4),
  (2, 5),
  (3, 4),
  (3, 5),
  (4, 5)],
 [(1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (1, 6),
  (2, 3),
  (2, 4),
  (2, 5),
  (2, 6),
  (3, 4),
  (3, 5),
  (3, 6),
  (4, 5),
  (4, 6),
  (5, 6)],
 [(1, 2, 3)

In [55]:
newFindD1([
 (1, 2, 4),
 (1, 2, 5),
 (1, 2, 6),
 (1, 3, 4),
 (1, 3, 5),
 (1, 3, 6),
 (1, 4, 5),
 (1, 4, 6),
 (1,5,6),
 (2,3,4),
 (2, 3, 5),
 (2, 3, 6),
 (2, 4, 5),
 (2, 4, 6),
 (2, 5, 6),
 (3, 4, 5),
 (3, 4, 6),
 (3, 5, 6),
 ],[],[])

[[1, 2, 3],
 [1, 4],
 [1, 5],
 [1, 6],
 [2, 4],
 [2, 5],
 [2, 6],
 [3, 4],
 [3, 5],
 [3, 6],
 [4, 5, 6]]

In [71]:
newFindD1([(1, 2),
  (1, 3),
  (2, 3),
  (4, 5),
  (1, 4, 6),
  (2, 5, 6),
  (1, 5, 6),
  (2, 4, 6),
  (3, 4, 6)],[],[])

[[1, 4], [1, 5], [1, 6], [2, 4], [2, 5], [2, 6], [3, 4], [3, 5, 6], [4, 6]]

In [43]:
checkD1([[1, 4], [1, 5, 6], [2, 4, 5], [2, 6], [3, 4, 6], [3, 5]])

True

In [49]:
arr = []

for comb in combinations(list(range(1,7)),3):
    arr.append(comb)

arr

[(1, 2, 3),
 (1, 2, 4),
 (1, 2, 5),
 (1, 2, 6),
 (1, 3, 4),
 (1, 3, 5),
 (1, 3, 6),
 (1, 4, 5),
 (1, 4, 6),
 (1, 5, 6),
 (2, 3, 4),
 (2, 3, 5),
 (2, 3, 6),
 (2, 4, 5),
 (2, 4, 6),
 (2, 5, 6),
 (3, 4, 5),
 (3, 4, 6),
 (3, 5, 6),
 (4, 5, 6)]

In [70]:
def createD2Graph(structure):
    D2 = []

    for P in structure:
        if len(P) == 2:
            D2.append(P)

    P = flatten(D2)

    G = nx.Graph()
    G.add_nodes_from(P)
    G.add_edges_from(D2)

    return G

In [76]:
graph = createD2Graph([(1,2),(2,3),(3,4),(1,3,5),(2,4,5)])

In [74]:
def newCheckD2(D2graph: nx.Graph):
    visited = []

    for node in D2graph.nodes:
        if node not in visited:
            componentNodes = 0
            edgeCount = 0
            stack = []
            stack.append(node)

            while len(stack) > 0:
                current = stack.pop()
                if current not in visited:
                    visited.append(current)
                    componentNodes += 1
                    for neighbour in D2graph.neighbors(current):
                        edgeCount += 1
                        if neighbour not in visited:
                            stack.append(neighbour)
            
            expectedEdges = componentNodes * (componentNodes - 1) / 2

            if expectedEdges != edgeCount / 2:
                return False

    return True

In [77]:
newCheckD2(graph)

False